# Gmsh Python API

In this notebook, I redraw the channel mesh using the Python API of Gmsh. It's supposed to be easier to debug and modify interactively.

In [ ]:
import gmsh

# Initialize Gmsh
gmsh.initialize()

# Set the geometry kernel to OpenCASCADE
# This is equivalent to SetFactory("OpenCASCADE");
gmsh.model.add("ratchet_channel")
occ = gmsh.model.occ

# --- 1. Parameters ---
W_TOTAL = 200
H_TOTAL = 100
h = 5
m = 3
l = 2 * h
N = 3
w = 2 * h
cl_outer = 3  # Outer boundary characteristic length
cl_inner = 1   # Channel characteristic length

# --- 2. Create the outer rectangle ---
outer_rectangle = occ.addRectangle(0, 0, 0, W_TOTAL, H_TOTAL)
occ.synchronize()



# --- 3. Create the ratchet "tool" shapes using a helper function ---
def create_ratchet(x_start, y_start, is_top):
    """
    Creates a ratchet surface tool.
    Returns the tag of the Plane Surface.
    """
    p_current = gmsh.model.occ.getMaxTag(0) + 1 # Get a fresh point tag
    
    # Starting points
    p1 = occ.addPoint(x_start, y_start, 0, cl_inner, p_current)
    y_offset = 2 * h if not is_top else -2 * h
    p2 = occ.addPoint(x_start, y_start + y_offset, 0, cl_inner, p_current + 1)

    points = [p1, p2]
    
    # Loop over teeth
    for i in range(N):
        y_direction = 1 if not is_top else -1
        # Use p_current for unique tags
        p_current = gmsh.model.occ.getMaxTag(0) + 1
        
        # Ratchet tooth corner points
        pt1 = occ.addPoint(x_start + m + i * l, y_start + y_offset, 0, cl_inner, p_current)
        pt2 = occ.addPoint(x_start + m + i * l, y_start + y_offset/2, 0, cl_inner, p_current + 1)
        points.extend([pt1, pt2])

    # End points of the ratchet
    p_current = gmsh.model.occ.getMaxTag(0) + 1
    end_x = x_start + m + N * l # Correctly use N instead of the loop variable i
    
    pt_end1 = occ.addPoint(end_x, y_start + y_offset, 0, cl_inner, p_current)
    pt_end2 = occ.addPoint(end_x + m, y_start + y_offset, 0, cl_inner, p_current+1)
    pt_end3 = occ.addPoint(end_x + m, y_start, 0, cl_inner, p_current+2)
    points.extend([pt_end1, pt_end2, pt_end3])

    # Create lines connecting all the points
    lines = []
    for i in range(len(points) - 1):
        lines.append(occ.addLine(points[i], points[i+1]))
    # Add the final line to close the loop
    lines.append(occ.addLine(points[-1], points[0]))
    
    # Create the surface
    cloop = occ.addCurveLoop(lines)
    surface = occ.addPlaneSurface([cloop])
    return surface

In [7]:
# Create the bottom and top ratchets
x01 = 0.5 * (W_TOTAL - N * l - 2 * m)
y01 = 0.5 * (H_TOTAL - 4 * h - w)
bottom_ratchet = create_ratchet(x01, y01, is_top=False)

x02 = x01
y02 = H_TOTAL - y01
top_ratchet = create_ratchet(x02, y02, is_top=True)

# --- 4. Perform the Boolean Operation ---
# Subtract the two ratchet tools from the outer rectangle
final_shape, _ = occ.cut([(2, outer_rectangle)], [(2, bottom_ratchet), (2, top_ratchet)], removeObject=True, removeTool=True)

# IMPORTANT: Synchronize the CAD kernel with the Gmsh model
occ.synchronize()

In [8]:
# --- 5. Identify Boundaries and Assign Physical Groups ---
final_surface_tag = final_shape[0][1]

boundary_curves = gmsh.model.getBoundary([(2, final_surface_tag)], combined=False)
boundary_curves_tags = [c[1] for c in boundary_curves]

left_curves, right_curves, bottom_curves, top_curves, wall_curves = [], [], [], [], []
tol = 1e-2

for curve_tag in boundary_curves_tags:
    # if curve_tag < 0:
    #     continue
    curve_tag = abs(curve_tag)
    bbox = gmsh.model.getBoundingBox(1, curve_tag)
    xmin, ymin, _, xmax, ymax, _ = bbox
    if abs(xmin - 0) < tol and abs(xmax - 0) < tol:
        left_curves.append(curve_tag)
    elif abs(xmin - W_TOTAL) < tol and abs(xmax - W_TOTAL) < tol:
        right_curves.append(curve_tag)
    elif abs(ymin - H_TOTAL) < tol and abs(ymax - H_TOTAL) < tol:
        top_curves.append(curve_tag)
    elif abs(ymin - 0) < tol and abs(ymax - 0) < tol:
        bottom_curves.append(curve_tag)
    else:
        wall_curves.append(curve_tag)

# Identify boundary points and assign cl_outer
boundary_curves = gmsh.model.getBoundary([(2, outer_rectangle)])
points = []
for curve in boundary_curves:
    boundary_points = gmsh.model.getBoundary([curve])
    for point in boundary_points:
        points.append(point)

gmsh.model.mesh.setSize(points, cl_outer)

occ.synchronize()

In [11]:
boundary_curves = gmsh.model.getBoundary([(2, final_surface_tag)], combined=False)

In [16]:
len(wall_curves)

22

In [17]:
len(boundary_curves)

26

In [ ]:
# Assign Physical Groups
gmsh.model.addPhysicalGroup(2, [final_surface_tag], 1, "domain")
gmsh.model.addPhysicalGroup(1, bottom_curves, 1, "bottom")   # Use these for the periodic map
gmsh.model.addPhysicalGroup(1, right_curves, 2, "right") # Use these for the periodic map
gmsh.model.addPhysicalGroup(1, top_curves, 3, "top")   # Use these for the periodic map
gmsh.model.addPhysicalGroup(1, left_curves, 4, "left") # Use these for the periodic map
gmsh.model.addPhysicalGroup(1, wall_curves, 5, "walls") # Inner walls

# --- 6. Apply Periodicity ---
# This is the API equivalent of 'Periodic Curve {right} = {left} Translate ...'
# The affine transform is [1,0,0, Tx, 0,1,0, Ty, 0,0,1, Tz, 0,0,0, 1]
# For x-periodicity, Tx=W_TOTAL, Ty=0, Tz=0
# x_translation = [1, 0, 0, W_TOTAL, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
# gmsh.model.mesh.setPeriodic(1, right_curves, left_curves, x_translation)
# y_translation = [1, 0, 0, 0, 0, 1, 0, H_TOTAL, 0, 0, 1, 0, 0, 0, 0, 1]
# gmsh.model.mesh.setPeriodic(1, top_curves, bottom_curves, y_translation)
# Note: The original file had top/bottom periodicity which is omitted here
# for clarity, but could be added with another setPeriodic call.

# --- 7. Generate and Save Mesh ---
gmsh.model.mesh.generate(2)
gmsh.write("mesh.msh")

# --- 8. Launch GUI for Visual Inspection (Optional) ---
# if '-nopopup' not in sys.argv:
#     gmsh.fltk.run()

gmsh.finalize()

Info    : Reconstructing periodicity for curve connection 29 - 28
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 20%] Meshing curve 8 (Line)
Info    : [ 20%] Meshing curve 9 (Line)
Info    : [ 20%] Meshing curve 10 (Line)
Info    : [ 30%] Meshing curve 11 (Line)
Info    : [ 30%] Meshing curve 12 (Line)
Info    : [ 40%] Meshing curve 13 (Line)
Info    : [ 40%] Meshing curve 14 (Line)
Info    : [ 40%] Meshing curve 15 (Line)
Info    : [ 50%] Meshing curve 16 (Line)
Info    : [ 50%] Meshing curve 17 (Line)
Info    : [ 60%] Meshing curve 18 (Line)
Info    : [ 60%] Meshing curve 19 (Line)
Info    : [ 60%] Meshing curve 20 (Line)
Info    : [ 70%] Meshing curve 21 (Line)
Info    : [ 70%] Meshing curve 22 (Line)
Info    : [ 70%] Meshing curve 23 (Line)
Info    : [ 80%] Meshing curve 24 (Line)
Info    : [ 80%] Meshing curve 25 (Line)
Info    : [ 90%] Meshing curve 26 (Line)
Info    : [ 9

# Square

In [19]:
import gmsh

gmsh.initialize()
gmsh.model.add("square")

# Parameters
W = 50.0   # length of the square
H = 50.0
cl = 1  # target mesh size

# Define corner points (tags are assigned automatically)
p1 = gmsh.model.geo.addPoint(0, 0, 0, cl)
p2 = gmsh.model.geo.addPoint(W, 0, 0, cl)
p3 = gmsh.model.geo.addPoint(W, H, 0, cl)
p4 = gmsh.model.geo.addPoint(0, H, 0, cl)

# Define lines
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

# Line loop and surface
loop = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
surface = gmsh.model.geo.addPlaneSurface([loop])

# Physical groups
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l1, l2, l3, l4], tag=1)  # boundary
gmsh.model.setPhysicalName(1, 1, "Boundary")

gmsh.model.addPhysicalGroup(2, [surface], tag=2)  # domain
gmsh.model.setPhysicalName(2, 2, "Domain")

# Mesh generation
gmsh.model.mesh.generate(2)
gmsh.write("square_50.msh")     # Save as msh2 by default

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.0057803s, CPU 0.006262s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0828398s, CPU 0.08359s)
Info    : 3018 nodes 6038 elements
Info    : Writing 'square_50.msh'...
Info    : Done writing 'square_50.msh'
